In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks/")

Mounted at /content/drive


##模型构造

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0217, -0.0238,  0.0977,  0.0105, -0.0148, -0.0565,  0.0079,  0.0878,
         -0.0212,  0.0044],
        [-0.0137, -0.0460,  0.2088,  0.0024, -0.1002, -0.1230,  0.0777,  0.0531,
         -0.0521, -0.0071]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20,256)
    self.out = nn.Linear(256,10)

  def forward(self,X):
    return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.3069,  0.0404,  0.0570,  0.0696, -0.0238, -0.1102,  0.1557,  0.0033,
          0.0809, -0.1023],
        [-0.2693,  0.0815,  0.0396,  0.0691, -0.1128, -0.1278,  0.1803, -0.0339,
          0.1488, -0.0816]], grad_fn=<AddmmBackward0>)

In [5]:
class MySequential(nn.Module):
  def __init__(self,*args):
    super().__init__()
    for block in args:
      self._modules[block] = block
  
  def forward(self,X):
    for block in self._modules.values():
      X = block(X)
    return X

net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.2273,  0.0501,  0.1580,  0.0608,  0.0894,  0.0297, -0.1431,  0.1127,
          0.0728,  0.0151],
        [ 0.0828, -0.1069,  0.2730, -0.0684,  0.2085,  0.0586, -0.0933,  0.1580,
          0.0857,  0.1061]], grad_fn=<AddmmBackward0>)

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

tensor(0.4475, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.3883, grad_fn=<SumBackward0>)

##参数管理

In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

tensor([[-0.1915],
        [-0.1985]], grad_fn=<AddmmBackward0>)

In [3]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1667,  0.2423, -0.1331,  0.3153, -0.2407, -0.0861,  0.0867, -0.2091]])), ('bias', tensor([-0.1093]))])


In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1093], requires_grad=True)
tensor([-0.1093])


In [5]:
net[2].weight.grad == None

True

In [6]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
net.state_dict()['2.bias'].data

tensor([-0.1093])

In [8]:
def block1():
  return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
  net = nn.Sequential()
  for i in range(4):
    net.add_module(f'block {i}',block1())
  return net

rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.2194],
        [0.2194]], grad_fn=<AddmmBackward0>)

In [9]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [10]:
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight,mean=0,std=0.01)
    nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([-0.0098, -0.0031, -0.0145,  0.0084]), tensor(0.))

In [11]:
def init_constant(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight,1)
    nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [12]:
def xavier(m):
  if type(m) == nn.Linear:
    nn.init.xavier_uniform_(m.weight)

def init_42(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight,42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.0562,  0.4395, -0.5450, -0.6095])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.0928, -0.0000, -0.0000,  7.9645],
        [ 0.0000,  8.4717,  8.6496,  0.0000]], grad_fn=<SliceBackward0>)

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000,  8.9645])

In [15]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


##自定义层

In [16]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
  def __init__(self):
    super().__init__()
  
  def forward(self,X):
    return X-X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [17]:
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())

Y = net(torch.rand(4,8))
Y.mean()

tensor(-9.3132e-10, grad_fn=<MeanBackward0>)

In [20]:
class MyLinear(nn.Module):
  def __init__(self,in_units,units):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(in_units,units))
    self.bias = nn.Parameter(torch.randn(units,))

  def forward(self,X):
    linear = torch.matmul(X,self.weight.data)+self.bias.data
    return F.relu(linear)

dense = MyLinear(5,3)
dense.weight

Parameter containing:
tensor([[-0.6002, -0.9161,  1.0158],
        [-1.1975,  0.2808,  1.7866],
        [-0.2140, -0.1898,  0.8119],
        [-0.4853, -0.6234,  0.4460],
        [ 0.8087, -0.5466,  1.0864]], requires_grad=True)

In [21]:
dense(torch.rand(2,5))

tensor([[0.0000, 0.0000, 3.3660],
        [0.0000, 0.3253, 2.6794]])

In [22]:
net = nn.Sequential(MyLinear(64,8),MyLinear(8,1))
net(torch.rand(2,64))

tensor([[ 2.3589],
        [10.1720]])

##读写文件

In [23]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x,'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [24]:
y = torch.zeros(4)
torch.save([x,y],'x-files')
x2,y2 = torch.load('x-files')
(x2,y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [26]:
mydict = {'x':x,'y':y}
torch.save(mydict,'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [27]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20,256)
    self.output = nn.Linear(256,10)

  def forward(self,x):
    return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.rand(size=(2,20))
Y = net(X)

In [28]:
torch.save(net.state_dict(),'mlp.params')

In [30]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [31]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])